In [77]:
%matplotlib inline
import requests as http
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import lib.lastfm as lastfm
import lib.curator as curator
#from lib.spotify import User
import time
import calendar
import datetime
import re
import unicodedata
import importlib
import requests as http
import json
import uuid
import redis as rd
import tornado
import pickle

In [2]:
redis = rd.StrictRedis(host='localhost', port=6379, db=0)
user_binary = redis.get('bornofawesomeness-obj')
user = pickle.loads(user_binary)

In [3]:
count = 0
artists = user.artists
cap = len(artists)
last_key = 'b333a19b2c0397e8e4c1224b49b3e7cd'
last_genre_method = 'artist.gettopTags'
last_base = 'http://ws.audioscrobbler.com/2.0/?method={0}&api_key={1}&format=json'
last_genre_url = last_base.format(last_genre_method, last_key) + '&autocorrect=1&artist='
def normalize_tags(tags, index):
    try:
        string = tags[index]['name']
        result = re.findall(r'([A-z])', string)
        return ''.join(result).lower()
    except IndexError:
        return None
genres = {}
start_time = time.clock()
for artist in artists:
    count += 1
    url = last_genre_url + artist
    url = url.replace(' ', '+')
    response = http.get(url, timeout=200).json()
    try:
        tags = response['toptags']['tag']
        genres[artist] = [
            normalize_tags(tags, 0),
            normalize_tags(tags, 1)
        ]
    except KeyError:
        genres[artist] = [None, None]

time.clock() - start_time, 'seconds'

(3.9620659999999996, 'seconds')

In [20]:
from tornado import ioloop
from tornado.httpclient import AsyncHTTPClient, HTTPRequest
genres = {}
left_to_process = len(user.artists)
def handle_response(response, genres):
    if response.error:
        print("Error:", response.error, response.effective_url)
    else:
        try:
            res_json = json.loads(response.body.decode('utf-8'))
            tags = res_json['toptags']['tag']
            genres[artist] = [
                normalize_tags(tags, 0),
                normalize_tags(tags, 1)
            ]
        except KeyError:
            genres[artist] = [None, None]
    global left_to_process
    left_to_process -= 1
    if left_to_process == 0:
        print(time.clock() - start_time, 'seconds')

http_client = AsyncHTTPClient()
start_time = time.clock()
for artist in user.artists:
    url = last_genre_url + artist
    url = url.replace(' ', '+')
    req = HTTPRequest(url, request_timeout=200)
    #http_client.fetch(req, handle_response)

In [53]:
class Count:
    def __init__(self):
        self.count = 0
    def inc(self):
        self.count += 1
    def dec(self):
        self.count -= 1

In [54]:
def handle_response(response, genres, url_map):
    artist = url_map[response.effective_url]
    print(artist)
    if response.error:
        print("Error:", response.error, response.effective_url)
    else:
        try:
            res_json = json.loads(response.body.decode('utf-8'))
            tags = res_json['toptags']['tag']
            genres[artist] = [
                normalize_tags(tags, 0),
                normalize_tags(tags, 1)
            ]
        except KeyError:
            genres[artist] = [None, None]


In [55]:
def async_batch_requests(uris, response_func, callback, timeout, **kwargs):
    left_to_process = 0
    http_client = AsyncHTTPClient()
    count = Count()
    def response_wrapper(response):
        response_func(response, **kwargs)
        count.dec()
        if count.count == 0:
            callback(**kwargs)
    for uri in uris:
        req = HTTPRequest(uri, request_timeout=timeout)
        count.inc()
        http_client.fetch(req, response_wrapper)

In [56]:
urls = []
genres = {}
url_map = {}
for artist in user.artists:
    url = last_genre_url + artist
    url = url.replace(' ', '+')
    urls.append(url)
    url_map[url] = artist
time_start = time.clock()
def callback(**kwargs):
    print(kwargs)
async_batch_requests(urls, handle_response, callback, 200, genres=genres, url_map=url_map)

Quavo
Drake
Miguel
Marian Hill
Sampha
Birdman
Red Hot Chili Peppers
Travis Scott
Nicki Minaj
Descemer Bueno
Gente De Zona
Enrique Iglesias
Jon Bellion
Skrillex
Birdy Nam Nam
A$AP Rocky
Lord Flacko
ScHoolboy Q
Kanye West
Theophilus London
Allan Kingdom
Paul McCartney
T.I.
Young Thug
Rae Sremmurd
Bones
Rod Stewart
Mark Ronson
Static Major
P.O.D.
Lil Wayne
GITCHII
Atlas Genius
Fabian Mazur
Xaphoon Jones
GANZ
Luude
Sean Kingston
Justin Timberlake
Kelly Clarkson
3 Doors Down
Disco Killerz
Liquid Todd
Jimmy Gnecco
Morgan Page
Lit
Incubus
Kaskade
Skylar Grey
Breathe Carolina
deadmau5
IZII
Sage The Gemini
Mike Williams
Alan Walker
Lenny Kravitz
Jack Ü
Machine Gun Kelly
Diplo
Russ
Just A Gent
Chief Keef
Imogen Heap
NGHTMRE
GRRL PAL
SLUMBERJACK
2 Chainz
BRKLYN
KLP
Borgeous
G-Eazy
Lenachka
tyDi
Rvssian
M.R.I.
DVBBS
Sean Paul
R3hab
Armin van Buuren
KSHMR
Lauren Evans
The Weeknd
JAY Z
Cake
Logic
Apex Rise
Green Day
ODESZA
Eminem
Gizzle
Chris Brown
Tory Lanez
Marc E. Bassy
Mac Miller
Snakehips
Alpha

In [73]:
import lib.lastfm as lastfm
import importlib
importlib.reload(lastfm)
history = []
def callback(history_df):
    history = history_df
    print('done')
lastfm = lastfm.Lastfm(name='dude0faw3', spotify=user.library, callback=callback)

done


In [75]:
history

[]

In [76]:
print('hello')

hello
